In [1]:
import pandas as pd
import numpy as np
from flask import Flask, request, render_template, redirect, url_for
app = Flask(__name__)

In [2]:
# location, industry and job variables for querying
# TODO: input variables from front-end

# industry code
industry = 89
industrySubmit = True

# job code
job = "Deck and Fishing Hands"
jobSubmit = False

# city for jobs
location = "ACT"
locationSubmit = True

In [3]:
# load data about jobs in each state
ACT = pd.read_csv("data/ACT_jobsalary.csv")
NSW = pd.read_csv("data/NSW_jobsalary.csv")
NT = pd.read_csv("data/NT_jobsalary.csv")
QLD = pd.read_csv("data/QLD_jobsalary.csv")
TAS = pd.read_csv("data/TAS_jobsalary.csv")
SA = pd.read_csv("data/SA_jobsalary.csv")
VIC = pd.read_csv("data/VIC_jobsalary.csv")

# load data about cost of living in cities
COL = pd.read_csv("data/COL.csv")

# load data about available jobs
jobs = pd.read_csv("data/laborjobs.csv")

# load data about availalbe industries
industries = pd.read_csv("data/industries.csv")


In [ ]:
# returns score for respective jobs
def rankJob(row, costOfLiving, w1, w2):
    salary = row["salary"]
    growth = row["projemp"]
    return w1*(salary-costOfLiving)+w2*growth

In [ ]:
# returns the top n entries in relation to loc, industry etc.
def getTopN(stateDF, costOfLiving, n, w1, w2):
    stateDF["score"] = [rankJob(row, costOfLiving, w1, w2) for index, row in stateDF.iterrows()]
    stateDF["rank"] = stateDF["score"].rank(ascending=False)
    return stateDF.sort_values(by="rank")[:n]

In [ ]:
# filters by industry
def filterByIndustry(industrycode, dataframe):
    return dataframe[dataframe["ocode"] 
                     // np.power(10, np.log10(dataframe["ocode"]).astype(int) - 1) == industrycode]


In [ ]:
# maps the string to the corresponding stateDF
def mapStateDF(string):
    if string == "ACT":
        return ACT
    elif string == "NSW":
        return NSW
    elif string == "QLD":
        return QLD
    elif string == "TAS":
        return TAS
    elif string == "SA":
        return SA
    elif string == "NT":
        return NT
    else:
        return VIC

In [ ]:
#TODO: check if industry is submitted
industryList = list(industries["occupation"])

if industrySubmit:
    # extract from field
#     industry = ""
    
    # find the list of jobs
    jobList = filterByIndustry(industry, jobs)
    jobNames = list(jobList["occupation"])
    # TODO: display jobNames to the user
    jobListCodes = list(jobList["ocode"])
    
    # retrieve the top 3 jobs in the selected industry for each city
    resultNo = 3
    w1 = 0.5
    w2 = 0.5
    ACT_top3 =  getTopN(filterByIndustry(industry, ACT), 1, resultNo, w1, w2).reset_index().to_json()
    NSW_top3 =  getTopN(filterByIndustry(industry, NSW), 1, resultNo, w1, w2).reset_index().to_json()
    QLD_top3 =  getTopN(filterByIndustry(industry, QLD), 1, resultNo, w1, w2).reset_index().to_json()
    TAS_top3 =  getTopN(filterByIndustry(industry, TAS), 1, resultNo, w1, w2).reset_index().to_json()
    SA_top3 = getTopN(filterByIndustry(industry, SA), 1, resultNo, w1, w2).reset_index().to_json()
    VIC_top3 = getTopN(filterByIndustry(industry, VIC), 1, resultNo, w1, w2).reset_index().to_json()

    # TODO: check if job selected
    if jobSubmit:
        # extract from field
#         job = ""
        
        # look for the job occupation
        jobOccupationCode = jobList[jobList["occupation"] == job].iat[0, 0]
        if locationSubmit:
            # set location
#             location = ""
            stateCode = COL[COL["City"] == location]
            stateDF = mapStateDF(location)
            selection = stateDF[stateDF["ocode"] == jobOccupationCode]
            selection = selection.reset_index()
            job_json = selection.to_json()
    else:
        # rank by state
        if locationSubmit:
            # retrieve the location df
#             location = ""
            # set the weights and no of results to display
            w1 = 0.5
            w2 = 0.5
            resultNo = 5

            stateCode = COL[COL["City"] == location]
            stateDF = mapStateDF(location)
            filtered = stateDF[stateDF["ocode"].isin(pd.Series(jobListCodes))]
            
            # costOfLiving = 1 because it's irrelevant for this ranking
            dfRanked = getTopN(filtered, 1, resultNo, w1, w2)
            dfRanked = dfRanked.reset_index()
            ranked_job_json = dfRanked.to_json()

/Users/Chinmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Chinmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        return redirect(url_for("mapSketch"))
    return render_template("index.html")

@app.route("/mapSketch", methods=['GET', 'POST'])
def mapSketch():
    return render_template("mapSketch.html")

In [ ]:
if __name__=='__main__':
    app.run(port=5008)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Sep/2019 12:50:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2019 12:50:23] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2019 12:50:23] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2019 12:50:23] "GET /images/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2019 12:50:23] "GET /images/left-arrow.png HTTP/1.1" 404 -
